In [ ]:
# ============================================================
# korea.kr 정책뉴스 "AI" 검색 결과 크롤러
# Google Colab 실행용 코드
# ============================================================

!pip -q install requests beautifulsoup4 pandas lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

BASE_URL = "https://www.korea.kr"
LIST_URL = "https://www.korea.kr/news/policyNewsList.do"

# ------------------------------------------------------------
# 검색 설정
# ------------------------------------------------------------
SEARCH_KEYWORD = "AI"
PAGE_COUNT = 3   # 몇 페이지까지 크롤링할지 (원하면 늘리세요)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

# ------------------------------------------------------------
# 1) 뉴스 목록 수집
# ------------------------------------------------------------

news_list = []

for page in range(1, PAGE_COUNT + 1):
    print(f"[페이지 {page}] 크롤링 중…")

    params = {
        "pageIndex": page,
        "searchCondition": "total",
        "searchKeyword": SEARCH_KEYWORD,
    }

    resp = requests.get(LIST_URL, params=params, headers=headers)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "lxml")

    # 뉴스 리스트 추출
    items = soup.select("ul.policyNewsList li a")
    if not items:
        print("[경고] 뉴스 리스트 항목을 찾지 못함. 구조가 바뀌었을 수 있음.")

    for a in items:
        title = a.get_text(strip=True)
        href  = a.get("href")
        if not href:
            continue

        full_url = BASE_URL + href
        news_list.append({"title": title, "link": full_url})

    time.sleep(1)

print(f"\n총 {len(news_list)}건 뉴스 링크 수집 완료!")
pd.DataFrame(news_list).head()


[페이지 1] 크롤링 중…
[경고] 뉴스 리스트 항목을 찾지 못함. 구조가 바뀌었을 수 있음.
[페이지 2] 크롤링 중…
[경고] 뉴스 리스트 항목을 찾지 못함. 구조가 바뀌었을 수 있음.
[페이지 3] 크롤링 중…
[경고] 뉴스 리스트 항목을 찾지 못함. 구조가 바뀌었을 수 있음.

총 0건 뉴스 링크 수집 완료!


""


In [ ]:
# ============================================================
# 1. 라이브러리 설치
# ============================================================
!pip install requests beautifulsoup4 pandas tqdm lxml

# ============================================================
# 2. 라이브러리 로드
# ============================================================
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

# ============================================================
# 3. 기본 설정
# ============================================================
BASE_URL = "https://www.korea.kr"
LIST_URL = "https://www.korea.kr/news/policyNewsList.do"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"
}

KEYWORD = "AI"
MAX_PAGE = 5   # 필요 시 증가

results = []

# ============================================================
# 4. 목록 페이지 크롤링
# ============================================================
for page in range(1, MAX_PAGE + 1):
    print(f"📄 페이지 {page} 수집 중...")

    params = {
        "pageIndex": page,
        "searchKeyword": KEYWORD
    }

    res = requests.get(LIST_URL, headers=HEADERS, params=params)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    items = soup.select("ul.list_type01 > li")
    print(f"  └ 발견된 기사 수: {len(items)}")

    if not items:
        break

    for item in items:
        title_tag = item.select_one("a")
        if not title_tag:
            continue

        title = title_tag.get_text(strip=True)
        link = BASE_URL + title_tag["href"]

        info = item.select_one(".info")
        date = info.select_one(".date").get_text(strip=True) if info else ""
        ministry = info.select_one(".organ").get_text(strip=True) if info else ""

        summary = item.select_one(".txt")
        summary = summary.get_text(strip=True) if summary else ""

        # ====================================================
        # 5. 상세 페이지 크롤링
        # ====================================================
        detail_res = requests.get(link, headers=HEADERS)
        detail_soup = BeautifulSoup(detail_res.text, "lxml")

        content = detail_soup.select_one(".view_txt")
        content_text = content.get_text("\n", strip=True) if content else ""

        results.append({
            "title": title,
            "date": date,
            "ministry": ministry,
            "summary": summary,
            "content": content_text,
            "url": link
        })

        time.sleep(0.7)

# ============================================================
# 6. DataFrame 생성
# ============================================================
df = pd.DataFrame(results)

print("\n✅ 수집 완료")
print("총 기사 수:", len(df))
display(df.head())

# ============================================================
# 7. CSV 저장
# ============================================================
file_name = "korea_policy_ai_news.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"📁 CSV 저장 완료: {file_name}")


📄 페이지 1 수집 중...
  └ 발견된 기사 수: 0

✅ 수집 완료
총 기사 수: 0


""


📁 CSV 저장 완료: korea_policy_ai_news.csv


In [ ]:
# =====================================
# 1. 라이브러리
# =====================================
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# =====================================
# 2. 정책뉴스 전체 수집
# =====================================
BASE_URL = "https://www.korea.kr/news/policyNewsList.do"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

all_rows = []

for page in range(1, 6):  # 1~5페이지만 예시 (늘려도 됨)
    params = {"pageIndex": page}
    res = requests.get(BASE_URL, headers=HEADERS, params=params, timeout=10)
    soup = BeautifulSoup(res.text, "html.parser")

    items = soup.select("ul.list_type01 > li")
    print(f"[PAGE {page}] 수집 기사 수:", len(items))

    for it in items:
        title = it.select_one("a").get_text(strip=True)
        link = "https://www.korea.kr" + it.select_one("a")["href"]
        date = it.select_one(".date").get_text(strip=True)

        # 기사 본문
        try:
            article = requests.get(link, headers=HEADERS, timeout=10)
            art_soup = BeautifulSoup(article.text, "html.parser")
            content = art_soup.select_one(".view_cont").get_text(" ", strip=True)
        except:
            content = ""

        all_rows.append({
            "title": title,
            "date": date,
            "link": link,
            "content": content
        })

    time.sleep(1)

df = pd.DataFrame(all_rows)
print("\n📄 전체 수집 기사 수:", len(df))

# =====================================
# 3. AI 키워드 필터링
# =====================================
AI_PATTERN = re.compile(
    r"\bAI\b|인공지능|artificial intelligence|지능정보|딥러닝|머신러닝",
    re.IGNORECASE
)

df_ai = df[
    df["title"].str.contains(AI_PATTERN, na=False) |
    df["content"].str.contains(AI_PATTERN, na=False)
]

print("🤖 AI 관련 정책 기사 수:", len(df_ai))

# =====================================
# 4. 결과 확인
# =====================================
df_ai[["date", "title", "link"]].head(10)


[PAGE 1] 수집 기사 수: 0
[PAGE 2] 수집 기사 수: 0
[PAGE 3] 수집 기사 수: 0
[PAGE 4] 수집 기사 수: 0
[PAGE 5] 수집 기사 수: 0

📄 전체 수집 기사 수: 0


KeyError: 'title'

In [ ]:
# =====================================================
# 0. 라이브러리
# =====================================================
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

BASE_LIST_URL = "https://www.korea.kr/news/policyNewsList.do"
BASE_ARTICLE_URL = "https://www.korea.kr"

results = []

# =====================================================
# 1. 정책뉴스 목록 수집
# =====================================================
for page in range(1, 6):  # 필요 시 늘려도 됨
    print(f"\n📄 페이지 {page} 수집 중...")
    try:
        res = requests.get(
            BASE_LIST_URL,
            headers=HEADERS,
            params={"pageIndex": page},
            timeout=10
        )
        soup = BeautifulSoup(res.text, "html.parser")
        items = soup.select("ul.list_type01 > li")
        print(f" → 기사 {len(items)}개 발견")
    except Exception as e:
        print(" ❌ 목록 페이지 오류:", e)
        continue

    for idx, it in enumerate(items):
        try:
            a_tag = it.select_one("a")
            title = a_tag.get_text(strip=True)
            link = BASE_ARTICLE_URL + a_tag["href"]
            date = it.select_one(".date").get_text(strip=True)
        except Exception as e:
            print("  ⚠️ 기본 정보 파싱 실패:", e)
            continue

        # =====================================================
        # 2. 기사 본문 수집 (안 깨지는 구조)
        # =====================================================
        content = ""
        try:
            art = requests.get(link, headers=HEADERS, timeout=10)
            art_soup = BeautifulSoup(art.text, "html.parser")

            # 시도 1
            cont = art_soup.select_one(".view_cont")
            if cont:
                content = cont.get_text(" ", strip=True)
            else:
                # 시도 2
                cont = art_soup.select_one("#cont")
                if cont:
                    content = cont.get_text(" ", strip=True)
                else:
                    content = ""

        except Exception as e:
            print(f"   ⚠️ 본문 수집 실패: {title[:30]}...")

        results.append({
            "date": date,
            "title": title,
            "link": link,
            "content": content
        })

        time.sleep(0.5)

# =====================================================
# 3. DataFrame 생성
# =====================================================
df = pd.DataFrame(results)
print("\n✅ 전체 수집 기사 수:", len(df))

# =====================================================
# 4. AI 키워드 필터링
# =====================================================
AI_REGEX = re.compile(
    r"\bAI\b|인공지능|artificial intelligence|딥러닝|머신러닝|지능정보",
    re.IGNORECASE
)

df_ai = df[
    df["title"].str.contains(AI_REGEX, na=False) |
    df["content"].str.contains(AI_REGEX, na=False)
]

print("🤖 AI 관련 정책 기사 수:", len(df_ai))

# =====================================================
# 5. 결과 확인
# =====================================================
df_ai[["date", "title", "link"]].head(10)



📄 페이지 1 수집 중...
 → 기사 0개 발견

📄 페이지 2 수집 중...
 → 기사 0개 발견

📄 페이지 3 수집 중...
 → 기사 0개 발견

📄 페이지 4 수집 중...
 → 기사 0개 발견

📄 페이지 5 수집 중...
 → 기사 0개 발견

✅ 전체 수집 기사 수: 0


KeyError: 'title'

In [ ]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import re

URL = "https://www.korea.kr/openapi/policyNewsList.do"

params = {
    "pageIndex": 1,
    "pageSize": 100
}

res = requests.get(URL, params=params, timeout=10)
print("HTTP 상태:", res.status_code)

root = ET.fromstring(res.text)

rows = []

for item in root.findall(".//item"):
    title = item.findtext("title", default="")
    content = item.findtext("content", default="")
    date = item.findtext("regDate", default="")
    link = item.findtext("url", default="")

    rows.append({
        "title": title,
        "date": date,
        "content": content,
        "link": link
    })

df = pd.DataFrame(rows)
print("📄 전체 기사 수:", len(df))

# ===========================
# AI 필터링
# ===========================
AI_REGEX = re.compile(
    r"\bAI\b|인공지능|artificial intelligence|딥러닝|머신러닝|지능정보",
    re.IGNORECASE
)

df_ai = df[
    df["title"].str.contains(AI_REGEX, na=False) |
    df["content"].str.contains(AI_REGEX, na=False)
]

print("🤖 AI 관련 정책 기사 수:", len(df_ai))
df_ai[["date", "title", "link"]].head()


HTTP 상태: 200


ParseError: not well-formed (invalid token): line 494, column 30 (<string>)

In [ ]:
/mnt/data/policy_news_mistral_summary.csv


NameError: name 'mnt' is not defined

In [ ]:
# ============================================================
# Policy News 본문 → Vector Embedding (Google Colab)
# ============================================================

# 0. 라이브러리 설치 (Colab 최초 1회)
!pip -q install sentence-transformers pandas numpy tqdm

# ------------------------------------------------------------
# 1. 라이브러리 로드
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ------------------------------------------------------------
# 2. 설정
# ------------------------------------------------------------
INPUT_CSV = "/mnt/data/policy_news_mistral_summary.csv"
OUTPUT_CSV = "/mnt/data/policy_news_with_embeddings.csv"
OUTPUT_NPY = "/mnt/data/policy_news_embeddings.npy"

TEXT_COLUMN = "DataContents"   # 본문 컬럼명
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# ↑ 한국어 지원 + 속도/성능 밸런스 우수

# ------------------------------------------------------------
# 3. 데이터 로드
# ------------------------------------------------------------
df = pd.read_csv(INPUT_CSV)

if TEXT_COLUMN not in df.columns:
    raise ValueError(f"본문 컬럼 '{TEXT_COLUMN}' 이 CSV에 없습니다. 현재 컬럼: {list(df.columns)}")

texts = (
    df[TEXT_COLUMN]
    .fillna("")
    .astype(str)
    .tolist()
)

print(f"[INFO] 총 문서 수: {len(texts)}")

# ------------------------------------------------------------
# 4. 임베딩 모델 로드
# ------------------------------------------------------------
model = SentenceTransformer(MODEL_NAME)

# ------------------------------------------------------------
# 5. 본문 → 벡터 변환
# ------------------------------------------------------------
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True  # cosine similarity 기준 사용 시 권장
)

print(f"[INFO] Embedding shape: {embeddings.shape}")

# ------------------------------------------------------------
# 6. 결과 저장
# ------------------------------------------------------------

# (1) CSV 저장 (벡터를 JSON 문자열로)
df["embedding"] = [
    json.dumps(vec.tolist(), ensure_ascii=False)
    for vec in embeddings
]
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

# (2) NPY 저장 (검색엔진/DB 적재용)
np.save(OUTPUT_NPY, embeddings)

print("[DONE]")
print(f"- CSV  : {OUTPUT_CSV}")
print(f"- NPY  : {OUTPUT_NPY}")


[INFO] 총 문서 수: 50


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO] Embedding shape: (50, 384)
[DONE]
- CSV  : /mnt/data/policy_news_with_embeddings.csv
- NPY  : /mnt/data/policy_news_embeddings.npy


In [ ]:
# ============================================================
# DataContents → QA Retrieval Vector Store (Google Colab)
# ============================================================

!pip -q install sentence-transformers pandas numpy tqdm faiss-cpu

# ------------------------------------------------------------
# 1. 라이브러리
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ------------------------------------------------------------
# 2. 경로 & 설정
# ------------------------------------------------------------
INPUT_CSV = "/mnt/data/policy_news_mistral_summary.csv"

META_CSV  = "/mnt/data/policy_news_chunks_meta.csv"
EMB_NPY   = "/mnt/data/policy_news_chunks_embeddings.npy"
FAISS_IDX = "/mnt/data/policy_news_faiss.index"

TEXT_COLUMN = "DataContents"
TITLE_COLUMN = "Title"

MODEL_NAME = "intfloat/multilingual-e5-base"

CHUNK_SIZE = 300     # 글자 기준
CHUNK_OVERLAP = 50   # 문맥 보존

# ------------------------------------------------------------
# 3. 텍스트 분할 함수
# ------------------------------------------------------------
def chunk_text(text, chunk_size=300, overlap=50):
    chunks = []
    start = 0
    text = text.strip()
    length = len(text)

    while start < length:
        end = start + chunk_size
        chunk = text[start:end].strip()

        if len(chunk) > 30:
            chunks.append(chunk)

        start = end - overlap

    return chunks

# ------------------------------------------------------------
# 4. CSV 로드
# ------------------------------------------------------------
df = pd.read_csv(INPUT_CSV)

if TEXT_COLUMN not in df.columns:
    raise ValueError(f"'{TEXT_COLUMN}' 컬럼이 없습니다.")

# ------------------------------------------------------------
# 5. 문서 → Chunk + 메타데이터
# ------------------------------------------------------------
rows = []
chunk_id = 0

for doc_id, row in tqdm(df.iterrows(), total=len(df)):
    body = str(row[TEXT_COLUMN])
    title = str(row[TITLE_COLUMN]) if TITLE_COLUMN in df.columns else ""

    for idx, chunk in enumerate(chunk_text(body)):
        rows.append({
            "chunk_id": chunk_id,
            "doc_id": doc_id,
            "chunk_index": idx,
            "title": title,
            "text": chunk
        })
        chunk_id += 1

chunk_df = pd.DataFrame(rows)
print(f"[INFO] 생성된 Chunk 수: {len(chunk_df)}")

# ------------------------------------------------------------
# 6. 임베딩 모델 로드
# ------------------------------------------------------------
model = SentenceTransformer(MODEL_NAME)

# e5 모델 규칙: passage prefix 필수
passages = [
    f"passage: {text}"
    for text in chunk_df["text"].tolist()
]

# ------------------------------------------------------------
# 7. Chunk → Vector
# ------------------------------------------------------------
embeddings = model.encode(
    passages,
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)

print(f"[INFO] Embedding shape: {embeddings.shape}")

# ------------------------------------------------------------
# 8. 저장
# ------------------------------------------------------------
chunk_df.to_csv(META_CSV, index=False, encoding="utf-8-sig")
np.save(EMB_NPY, embeddings)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine similarity
index.add(embeddings)
faiss.write_index(index, FAISS_IDX)

print("[DONE]")
print(f"- META   : {META_CSV}")
print(f"- VECTOR : {EMB_NPY}")
print(f"- FAISS  : {FAISS_IDX}")


100%|██████████| 50/50 [00:00<00:00, 9345.60it/s]

[INFO] 생성된 Chunk 수: 334


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

[INFO] Embedding shape: (334, 768)
[DONE]
- META   : /mnt/data/policy_news_chunks_meta.csv
- VECTOR : /mnt/data/policy_news_chunks_embeddings.npy
- FAISS  : /mnt/data/policy_news_faiss.index
